
<font color="#1874CD"><h1 align="left">**Datamecum**</h1></font>
<font color="#6E6E6E"><h2 align="left">**Árboles de decisión**</h2></font>
<font color="#6E6E6E"><h3 align="left">Notebook: aplicación de árboles de decisión en un problema de clasificación</h3></font>
***

El objetivo de este *notebook* es aplicar los árboles de decisión a un problema de clasificación. 

Para ello, vamos a considerar un problema de *scoring* de crédito para determinar qué clientes de los que piden un préstamo tienen riesgo de impago, es decir, van a causar mora.

In [1]:
# ------------------------------------------------------------------------------
# Librerías
# ------------------------------------------------------------------------------
import pandas as pd
import numpy as np
import seaborn as sns
from IPython.display import display
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, roc_curve, auc
from sklearn.tree import export_text
from sklearn.metrics import ConfusionMatrixDisplay

import math

%matplotlib inline

# ------------------------------------------------------------------------------
# Configuración warnings
# ------------------------------------------------------------------------------
import warnings
warnings.filterwarnings('once')

In [ ]:
## Cargamos los datos
data = pd.read_csv('./data/cs-construction.csv')

data.head(5)

## Tenemos una columna que etiqueta el número de fila, la eliminamos desde lectura de fichero
## Aprovechamos para eliminar - en los nombres de las columnas

In [ ]:
## Cargamos los datos
## Eliminamos la primera columna (con la etiqueta "Unnamed") que es sólo el número de fila
data = pd.read_csv('./data/cs-construction.csv').drop('Unnamed: 0', axis = 1) ## axis = 1 es para indicar que busque por columnas!

## List comprehension: creamos lista a través de un for para eliminar - en los nombres de las columnas
data.columns = [x.replace('-', '') for x in data.columns]

data.head(5)

In [ ]:
print("Dimensiones:\n")
print(data.shape)

El diccionario de los datos es el siguiente.

In [ ]:
description = pd.read_excel('./data/cs-DataDictionary.xls', header=1).set_index("Variable Name")
description

In [ ]:
## En pandas hay límites en la visualización
pd.set_option('display.max_colwidth', None)

description

## Análisis exploratorio de los datos (EDA)

En primer lugar, para hacernos una idea de qué datos disponemos calculamos algunas estadísticas para ver la existencia de valores ausentes o el rango de las variables (aunque para árboles de decisión no es necesario normalizar).

In [ ]:
## Obtenemos algunas estadísticas
data.describe().T

## La variable a predecir es SeriousDlqin2yrs que es una binaria 0 o 1, donde 1 indica que el cliente ha hecho mora

In [ ]:
## Visualizamos valores ausentes
data.isna().sum()

## isna() -> devuelve True (1) o False (0) según si hay valor ausente. Sumando, obtenemos el total.

En el *describe* anterior podemos destacar los siguientes puntos:

* Edad. Tenemos personas con 109 años y 0 años pidiendo un crédito (*outliers*). Tendremos que limpiarlos de la base de datos.
* Ingreso mensual. Tenemos valores ausentes. Por lo tanto, podrían imputarse valores o eliminarlos dependiendo del problema. En este caso concreto, es una variable imprescindible para la detección del riesgo de crédito, por lo que no sería bueno tener valores imputados.
* *Target*. Al ser binaria, la media nos indica que tenemos aproximadamente un 7% de 1s (clientes que han causado mora), es decir, tenemos un problema desbalanceado.
* Número de retrasos entre 1 y 2 meses. El máximo es 98, por lo que no tiene sentido. Probablemente sea la imputación de un *missing*, para indicar que no se sabe el dato.

Ahora, fijamos *target* y variables predictoras.

In [ ]:
## Identificamos target y variables predictoras
class_column = 'SeriousDlqin2yrs'
class_names = ['no financial distress', 'financial distress']
attribute_columns = list(data.columns)
attribute_columns.remove(class_column)

print(f"Target: {class_column}")
print(f"Variables explicativas: {attribute_columns}")

Ahora sería interesante ver con qué proporción de 1s y 0s contamos.

In [ ]:
## Esta función nos ayuda a formatear el gráfico: añadimos título y %s
def add_freq(data):
    ncount = len(data)

    ax2=ax.twinx()

    ax2.yaxis.tick_left()
    ax.yaxis.tick_right()

    ax.yaxis.set_label_position('right')
    ax2.yaxis.set_label_position('left')
    ax2.set_ylabel('Class Frequency (%)')

    for p in ax.patches:
        x=p.get_bbox().get_points()[:,0]
        y=p.get_bbox().get_points()[1,1]
        ax.annotate('{:.1f}%'.format(100.*y/ncount), (x.mean(), y), 
                ha='center', va='bottom')

    ax2.set_ylim(0,100)
    ax2.grid(None)

## Pintamos con seaborn la proporciones
ax = sns.countplot(x = data.SeriousDlqin2yrs ,palette="Set3")
sns.set(font_scale=1.5)
ax.set_xlabel(' ')
ax.set_ylabel(' ')
fig = plt.gcf()
fig.set_size_inches(7,5)
ax.set_ylim(top=160000)
ax.set_xticklabels(["0 ({0})".format(class_names[0]), "1 ({0})".format(class_names[1])])
ax.set_title(class_column)

## Añadimos el formato
add_freq(data)

Tenemos una alta proporción de clientes que no causan impago. Por lo tanto, vemos que es un problema **altamente desbalanceado**, es decir, tenemos muchos más ejemplos de una clase que de otra.

Esto es un problema porque puede hacer que el modelo tienda a predecir que todos los clientes pertenecen a una clase. En este caso de uso, si el modelo dijera que todos son 0s, el *accuracy* de este modelo sería de un 93.3%. Esta métrica nos parecería realmente buena, pero no es así realmente porque está prediciendo una única clase. Esta métrica sería nuestro *baseline*, es decir, el valor que tenemos que mejorar para lograr tener detección.

Ahora, visualizamos el conjunto de datos total.

In [ ]:
data.hist(figsize=(15,15));

Muchas variables poseen una distribución asimétrica positiva. Notar, que aunque no se muestre la cola, es porque el valor 0 tiene mucho peso.

In [ ]:
## Aquí observamos esa tendencia. Valores 98 y 96 son outliers
data["NumberOfTime6089DaysPastDueNotWorse"].value_counts()

### Limpieza de los datos: *outliers*

Eliminamos algunos *outliers* para tener los datos más limpios y volveremos a graficar las distribuciones.

In [ ]:
data = data[(data["age"]>=18) & (data["age"]<=80)]
data = data[data["NumberOfTime3059DaysPastDueNotWorse"]<96]
data = data[data["NumberOfTime6089DaysPastDueNotWorse"]<96]
data = data[data["NumberOfTimes90DaysLate"]<96]

data.describe().T[["count", "min", "max", "mean", "std"]]

In [ ]:
fontsize = 10
fig = data[attribute_columns].hist(bins = 20, figsize = (12, 12),
                                   xlabelsize=fontsize,
                                   ylabelsize=fontsize)
for x in fig.ravel():
    x.title.set_size(fontsize)

### Limpieza de los datos: valores ausentes

Con los valores ausentes podemos aplicar diferentes estrategias:

* **Eliminarlos**. Esto dependerá del número de muestras que tengamos: si tenemos suficientes, podemos eliminar al no tener un dato real.
* **Imputarles un valor** como la media o mediana, pero cuidado, porque esto genera sesgos, al imputar un dato que no sabemos. Otros métodos es la imputación a través de un modelo, es decir, predecir el valor de la variable con datos ausentes en función del resto de variables.

En *scikit-learn* no deben de haber variables con datos ausentes.

Por otro lado, destacar que se podría mirar a qué grupo pertenecen estos datos ausentes para evitar introducir sesgos. Por ejemplo, ¿todos los *missings* pertenecen a clientes jóvenes? ¿o de una ciudad en concreto?

En este caso, vamos a eliminarlos al tener suficientes muestras y ser la variable de ingreso mensual tan imprescindible y con riesgo de crear sesgos si imputamos.

In [ ]:
## Continuamos teniendo valores ausentes
data.isnull().sum()

In [ ]:
## Eliminamos valores ausentes
clean_data = data.copy()
clean_data.dropna(inplace=True)
clean_data.isnull().sum()

In [21]:
## Volvemos a definir vector de nombres
attribute_names = list(clean_data.columns)
attribute_names.remove(class_column)

In [ ]:
fontsize = 10
fig = clean_data[attribute_names].hist(bins = 20, figsize = (12, 12),
                                       xlabelsize=fontsize,
                                       ylabelsize=fontsize)
for x in fig.ravel():
    x.title.set_size(fontsize)

## Construcción del modelo

Una vez se han explorado los datos y preprocesado, el siguiente paso es construir el modelo. Primeramente, vamos a dividir los datos en tres conjuntos:

* **Entrenamiento**. Serán las muestras que consideremos para la construcción del modelo.
* **Validación**. Serán las muestras que usemos para encontrar los mejores hiperparámetros.
* **Test**. Serán las muestras que el modelo no vea en ningún momento para determinar el rendimiento real del modelo.

In [ ]:
## ------------- ##
## -- Forma 1 -- ##
## ------------- ##

## División en entrenamiento, validación y test

## Entrenamiento 55%, validación 15% y test 30%
df_train, df_val = train_test_split(
    clean_data,
    random_state = 10,
    test_size=0.45
    )

## División en entrenamiento y validación
df_val, df_test = train_test_split(
    df_val,
    random_state = 10,
    test_size=(2/3) ## Del 45% restante, nos quedamos con el 66% que equivale al 30% para test
    )

## Dividimos X e y
y_train = df_train[class_column]
y_val = df_val[class_column]
y_test = df_test[class_column]

X_train = df_train[attribute_columns]
X_val = df_val[attribute_columns]
X_test = df_test[attribute_columns]

## Comprobamos que la partición ha ido bien
print(f"Dimensiones train: {X_train.shape}.")
print(f"Dimensiones val: {X_val.shape}.")
print(f"Dimensiones test: {X_test.shape}.")

In [24]:
## ------------- ##
## -- Forma 2 -- ##
## ------------- ##

## Función que nos permite hacer la partición en tres conjuntos de forma directa
def train_val_test_split(X, y, val_size=0.0, test_size=0.8, random_state=None):
    
    if isinstance(val_size, int):
        nval = val_size
    else:
        nval = int(len(X)*val_size)
    if isinstance(test_size, int):
        ntest = test_size
    else:
        ntest = int(len(X)*test_size)
    
    X_trval, X_test, y_trval, y_test = train_test_split(X, y,
                                                        test_size=ntest,
                                                        random_state=random_state)
    if random_state == None:
        aux = None
    else:
        aux = 54773*random_state
    X_train, X_val, y_train, y_val = train_test_split(X_trval, y_trval,
                                                      test_size=nval,
                                                      random_state=aux)

    return X_train, X_val, X_test, y_train, y_val, y_test

In [ ]:
## División en entrenamiento, validación y test 

y = np.array(clean_data[class_column])
X = np.array(clean_data[attribute_columns])

aux = train_val_test_split(X, y, val_size=.15, test_size=0.3, random_state=10)
X_train, X_val, X_test, y_train, y_val, y_test = aux

## Comprobamos que la partición ha ido bien
print(f"Dimensiones train: {X_train.shape}.")
print(f"Dimensiones val: {X_val.shape}.")
print(f"Dimensiones test: {X_test.shape}.")

In [27]:
## Si queremos usar la siguiente funcionalidad con la partición de la forma 2,
## tenemos que pasar de un array a un df, porque iteramos por columnas del df
X_train = pd.DataFrame(X_train)
X_train.columns = attribute_columns

X_val = pd.DataFrame(X_val)
X_val.columns = attribute_columns

X_test = pd.DataFrame(X_test)
X_test.columns = attribute_columns

In [ ]:
## Nos aseguramos que los tres conjuntos tienen la misma distribución

fig_cols = 2
fig_rows = math.ceil(len(attribute_columns)/fig_cols)
fig = plt.figure(figsize=(12, math.ceil(7*fig_rows/3)))

for idx, col in enumerate(attribute_columns):
    ax = fig.add_subplot(fig_rows, fig_cols, idx + 1)
    ax.hist(X_train[col], bins = 40, density = True, alpha = 0.5, label = "Train")
    ax.hist(X_test[col], bins = 40, density = True, alpha = 0.5, label = "Test")
    ax.hist(X_val[col], bins = 40, density = True, alpha = 0.5, label = "Val")
    ax.legend(loc = "upper right")
    ax.set_title(col)
fig.tight_layout()
plt.show()

In [ ]:
model = DecisionTreeClassifier(max_depth=3, ## máxima profundidad
                               min_samples_split=200,  ## número mínimo de ejemplos para dividir
                               min_samples_leaf=100) ## número mínimo de ejemplos que tiene que llegar a una hoja

print(model)

model.fit(X_train, y_train)

In [ ]:
plt.figure(figsize=(25,15), dpi=200)
plot_tree(model, rounded=True, 
          proportion=True, 
          label='none',
          feature_names=attribute_columns,
          class_names=class_names,
          filled=True);

## Respuesta a la pregunta: si es que no, hacia la derecha
## Respuesta a la pregunta: si es que sí, hacia la izquierda

In [ ]:
## Calidad predictiva del modelo

print("Accuracy training = {}".format(model.score(X_train, y_train)))
print("Accuracy test = {}".format(model.score(X_test, y_test)))

## Calculamos estadístico AUC en train y test
print(f"\nAUC en train: {roc_auc_score(y_train, model.predict_proba(X_train)[:, 1])}")
print(f"AUC en test: {roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])}")

## Calculamos también precisión y recall en test
print(f"\nPrecisión en test: {precision_score(y_test, model.predict(X_test))}")
print(f"Recall en test: {recall_score(y_test, model.predict(X_test))}")

# Represenetación de la matriz de confusión
confusion_matrix_test = confusion_matrix(y_test, model.predict(X_test))
print(confusion_matrix_test) # row: real class; column: predicted class

plt.figure(figsize=(4,4))
sns.heatmap(confusion_matrix_test, cmap=plt.cm.rainbow)
plt.title("Matriz de confusión en test")
tick_marks = class_names
plt.ylabel('Clase real')
plt.xlabel('Clase predicha');

In [ ]:
print(classification_report(y_test, model.predict(X_test)))

El *recall* no es lo suficientemente bueno, ya que solo detecta el 12% del impago. Sin embargo, el modelo proporciona segmentación de clientes para, por ejemplo, establecer grupos de riesgo.

In [ ]:
## Funcionalidad predict: devuelve la clase predicha
model.predict(X_test)[:20]

In [ ]:
## Funcionalidad predict_proba: devuelve la predicción a cada clase
model.predict_proba(X_test)[:20]

In [ ]:
## Si calculamos cuántas probabilidades únicas tenemos, ¿cuántas hay?
np.unique(model.predict_proba(X_test)[:,1])

In [ ]:
## Caso de uso: ordenar a través de la probabilidad para tener una ordenación de clientes propensos a hacer un impago
df_aux = pd.DataFrame({"P_default": model.predict_proba(X_test)[:,1], "real class":y_test})
df_aux.sort_values("P_default", ascending=False)[:20]

Ahora, dibujamos la curva ROC para obtener el área bajo la curva:

* Cuánto más cercano a 1, mejor.
* Si el modelo fuera constante o aleatorio, la curva ROC estaría en la diagonal.

In [ ]:
target_class = "financial distress"

positive_class = class_names.index(target_class)

y_test_proba_predict  = model.predict_proba(X_test)

fpr, tpr, thresholds = roc_curve(y_test,
                                 y_test_proba_predict[:,positive_class],
                                 pos_label=positive_class)
roc_auc = auc(fpr, tpr)

plt.plot(fpr, tpr, "-o", color='darkorange',
         lw=2, label='model (AUC = {:.2f})'.format(roc_auc))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label = 'random')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC at test, target = '+'"'+target_class+'"')
plt.legend(loc="lower right");

In [39]:
## Reglas
def get_rules_from_tree(clf, feature_names, class_values, X, y):
    from sklearn.tree._tree import TREE_LEAF
    
    rules = {}
    
    def recurse(node_id, aux_rules=[]):
        left = clf.tree_.children_left[node_id]
        right = clf.tree_.children_right[node_id]

        ## Check si es un nodo de decisión
        if left != TREE_LEAF:
            feature = feature_names[clf.tree_.feature[node_id]]
            new_rule = "(" + feature + " {0} " + "%.4f" % clf.tree_.threshold[node_id] + ")"
            recurse(left, aux_rules + [new_rule.format('<=')])
            recurse(right, aux_rules + [new_rule.format('>')])
            return
        else: # Leaf
            rules[node_id] = " and ".join(aux_rules)
            return

    recurse(node_id=0)
    classification_leafs = clf.apply(X)
    extended_rules = {}
    for k,v in rules.items():
        aux1 = y[classification_leafs == k].tolist()
        aux2 = []
        for c, name in zip(clf.classes_, class_values):
            aux3 = aux1.count(c)
            aux2.append((name, aux3, aux3/len(aux1)))
        aux_dict = {}
        extended_rules[k] = [v, aux2]
    
    return extended_rules

In [ ]:
## Reglas en train
classes_ = model.classes_.tolist()

## Estadísticas medidas en el conjunto de training:
rules = get_rules_from_tree(model, attribute_columns, classes_, X_train, y_train)

target_class = 1

## Ordenadas por probabilidad de que clase=1 (credit default):
print(len(rules), "reglas\n")
for item in sorted(rules.items(),
                   key=lambda r: r[1][1][classes_.index(target_class)][2],
                   reverse=True):
    print(item)
    print("   Probabilidad clase={}: {}".format(target_class, item[-1][-1][-1][-1]))
    print("\n")

In [ ]:
## Reglas en validación (podemos extraer estadísticas más fiables, al no haber entrado en entrenamiento)
rules = get_rules_from_tree(model, attribute_columns, classes_, X_val, y_val)

target_class = 1

## Ordenadas por probabilidad de que clase=1 (credit default):
print(len(rules), "reglas\n")
for item in sorted(rules.items(),
                   key=lambda r: r[1][1][classes_.index(target_class)][2],
                   reverse=True):
    print(item)
    print("   Probabilidad clase={}: {}".format(target_class, item[-1][-1][-1][-1]))
    print("\n")

## Optimización de hiperparámetros

En esta parte, el objetivo es encontrar los mejores hiperparámetros que mejor calidad predictiva nos aporte.

Para ello, vamos a hacer un *grid search* con validación cruzada (*GridSearchCV*) que funciona de la siguiente manera: entrenamos X modelos (*cv*) modelos con diferentes combinaciones de los hiperparámetros que le pasemos (*param_grid*). En el ejemplo, sería:

* Consideramos 39 posibles valores para *max_leaf_nodes*.
* Consideramos 2 posibles valores para *max_depth*.
* Entrenaremos cada combinación 10 veces (parámetro *cv*): se tomará un subconjunto de los datos como entrenamiento y otro como validación distintos en cada repetición.

Con esto entrenamos 780 modelos (39·2·10), de tal forma que **obtengamos aquel que tenga la combinación que nos proporcione la mejor métrica AUC**. Destacar que no ponemos *accuracy* al ser un problema altamente desbalanceado y puede "engañarnos" en cuanto al rendimiento del modelo.

In [ ]:
param_grid = {"max_leaf_nodes": range(2, 41), ## número máximo de nodos hojas: entre 2 y 40
              "max_depth": range(2, 4)} ## máxima profundidad: entre 2 y 3

grid = GridSearchCV(DecisionTreeClassifier(), 
                    param_grid, 
                    cv=10,
                    verbose=3, 
                    scoring="roc_auc")

grid.fit(X_train, y_train)

In [ ]:
## Vemos cuál es el mejor modelo
grid.best_estimator_

In [45]:
## Lo guardamos para usarlo
model_gridsearch = grid.best_estimator_

In [ ]:
## Calidad predictiva del modelo

print("Accuracy training = {}".format(model_gridsearch.score(X_train, y_train)))
print("Accuracy test = {}".format(model_gridsearch.score(X_test, y_test)))

## Calculamos estadístico AUC en train y test
print(f"\nAUC en train: {roc_auc_score(y_train, model_gridsearch.predict_proba(X_train)[:, 1])}")
print(f"AUC en test: {roc_auc_score(y_test, model_gridsearch.predict_proba(X_test)[:, 1])}")

## Calculamos también precisión y recall en test
print(f"\nPrecisión en test: {precision_score(y_test, model_gridsearch.predict(X_test))}")
print(f"Recall en test: {recall_score(y_test, model_gridsearch.predict(X_test))}")

# Represenetación de la matriz de confusión
confusion_matrix_test = confusion_matrix(y_test, model_gridsearch.predict(X_test))
print(confusion_matrix_test) # row: real class; column: predicted class

plt.figure(figsize=(4,4))
sns.heatmap(confusion_matrix_test, cmap=plt.cm.rainbow)
plt.title("Matriz de confusión en test")
tick_marks = class_names
plt.ylabel('Clase real')
plt.xlabel('Clase predicha');

In [ ]:
target_class = "financial distress"

positive_class = class_names.index(target_class)

y_test_proba_predict  = model_gridsearch.predict_proba(X_test)

fpr, tpr, thresholds = roc_curve(y_test,
                                 y_test_proba_predict[:,positive_class],
                                 pos_label=positive_class)
roc_auc = auc(fpr, tpr)

plt.plot(fpr, tpr, "-o", color='darkorange',
         lw=2, label='model (AUC = {:.2f})'.format(roc_auc))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label = 'random')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC at test, target = '+'"'+target_class+'"')
plt.legend(loc="lower right");

## Importancia de los predictores

Los árboles de decisión también proporcionan la importancia de los predictores. En el caso de clasificación, la importancia de cada predictor en el modelo se calcula como la reducción total (normalizada) en el criterio de división, en este caso el índice Gini, que consigue el predictor en las divisiones en las que participa. Si un predictor no ha sido seleccionado en ninguna divisón, no se ha incluido en el modelo y por lo tanto su importancia es 0. 

In [ ]:
importancia_predictores = pd.DataFrame(
                            {'predictor': attribute_columns,
                             'importancia': model_gridsearch.feature_importances_}
                            )

print("Importancia de los predictores en el modelo")
print("")

importancia_predictores.sort_values('importancia', ascending=False)

## Guardado del modelo

Cuando ya tenemos un modelo lo suficientemente bueno, podemos guardarlo para su posterior uso.

In [50]:
import pickle

## Grabaría cuatro objetos (dump: meter dentro del fichero)

# with open("credit_scoring_tree.pkl", 'wb') as f:
   # pickle.dump(class_column, f)
   # pickle.dump(class_names, f)
   # pickle.dump(attribute_columns, f)
   # pickle.dump(model, f)

In [ ]:
## Para recuperar el guardado se hace en el mismo orden que se guardó
# with open("credit_scoring_tree.pkl", 'rb') as f:
   # class_column = pickle.load(f)
   # class_names = pickle.load(f)
   # attribute_columns = pickle.load(f)
   # model = pickle.load(f)
   
## Esto es tedioso, tenemos que saber el orden y al mínimo cambio nos fallará esta parte

In [51]:
## Más elegante y limpio: un diccionario

data = {
    "class_column": class_column,
    "class_names":  class_names,
    "attributes_names": attribute_columns,
    "modelo": model,
    "comentarios": "versión del 26 de enero de 2024"
}

with open("credit_scoring_tree.pkl", 'wb') as f:
    pickle.dump(data, f)

In [52]:
## Si lo queremos explotar en producción:

with open("credit_scoring_tree.pkl", 'rb') as f:
    data2 = pickle.load(f)

In [ ]:
## Al leerlo todo, llamando a data2 podemos ver qué objetos hay guardados del diccionario
data2

In [ ]:
## Recuperación de lo que se ha guardado para su uso

class_column     = data2["class_column"]
class_names      = data2["class_names"]
attributes_names = data2["attributes_names"]
model            = data2["modelo"]
print(data2["comentarios"])